[Following this:](https://pennylane.ai/qml/demos/how_to_catalyst_lightning_gpu)



In [ ]:
pip install pennylane pennylane-catalyst custatevec_cu12 pennylane-lightning-gpu jupyter

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4110170564.py, line 1)

In [1]:
import pennylane as qml
import jax.numpy as jnp

import jax

# Set number of wires
num_wires = 28

# Set a random seed
key = jax.random.PRNGKey(0)

dev = qml.device("lightning.gpu", wires=num_wires)

@qml.qjit(autograph=True)
@qml.qnode(dev)
def circuit(params):

    # Apply layers of RZ and RY rotations
    for i in range(num_wires):
        qml.RZ(params[3*i], wires=[i])
        qml.RY(params[3*i+1], wires=[i])
        qml.RZ(params[3*i+2], wires=[i])

    return qml.expval(qml.PauliZ(0) + qml.PauliZ(num_wires-1))

# Initialize the weights
weights = jax.random.uniform(key, shape=(3 * num_wires,), dtype=jnp.float32)

circuit(weights)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Array(1.65765899, dtype=float64)

In [3]:
@qml.qjit(autograph=True)
def workflow(params):
    g = qml.grad(circuit)
    return g(params)

workflow(weights)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('ag__',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('ag__',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


RuntimeError: KeyboardInterrupt (SIGINT)

In [ ]:
import catalyst
import optax

opt = optax.sgd(learning_rate=0.4)

def update_step(i, params, opt_state):
    energy, grads = catalyst.value_and_grad(circuit)(params)
    updates, opt_state = opt.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    catalyst.debug.print("Step = {i}", i=i)
    return (params, opt_state)

@qml.qjit
def optimization(params):
    opt_state = opt.init(params)
    (params, opt_state) = qml.for_loop(0, 10, 1)(update_step)(params, opt_state)
    return params

optimization(weights)